## Importando bibliotecas

In [17]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

Abertura de uma sessão utilizando a biblioteca requests_html e fornecendo o endereço que iremos realizar o scraper. 

In [18]:
session = HTMLSession()
session.cookies.set("COOKIE_NAME", "the cookie works", domain="https://www.goodreads.com/")

url = 'https://www.goodreads.com/shelf/show/'

genre = input('Type the genre you want to search (you can find the name on goodreads): ')

Para obter as notas e o ano de publicação, utilizei Regex, para identificar a média de notas, utilizando o método search.

In [19]:
def get_rating(input_text):
    match = re.search(r'avg rating (\d+\.\d+)', input_text)
    if match:
        avg_rating = float(match.group(1))
        return avg_rating

In [20]:
def get_year(input_text):
    match_published = re.search(r'published (\d{4})', input_text)
    if match_published:
        year_published = int(match_published.group(1))
        return year_published

### Variáveis que armazenarão os valores

In [21]:
titles = []
authors = []
ratings = []
released = []

A função get_data tem o papel de abrir uma sessão na url, passar as informações capturadas nela para o beautiful soup que analisará o HTML, sendo possível extrair dados do mesmo. Através da escolha de atributos e nomes de classes, retiramos o título, o autor, nota e ano. Cada um desses dados, será adicionado as listas declaradas acima. 

In [22]:
def get_data(url):
    r = session.get(url + genre)
    r.cookies
    soup = BeautifulSoup(r.text, 'html.parser')
    for d in soup.find_all('div', attrs={'class': 'elementList'}):
        info = d.find('div', attrs={'class', 'left'})
        if (info is not None):
            title = info.find('img', alt=True)
            titles.append(title.get('alt'))
            author = info.find('a', attrs={'class', 'authorName'})
            authors.append(author.find('span').text)
            smallText = info.find('span', attrs={'class', 'smallText'})
            rating = get_rating(smallText.text)
            year = get_year(smallText.text)
            ratings.append(rating)
            released.append(year)

    return

get_data(url)

In [23]:
released = np.array(released) # int

Como ponto final, escolhi criar um dataframe a partir desses dados, utilizando pandas.

In [24]:
df = pd.DataFrame({'Titulo': titles, 'Autor': authors, 'Nota': ratings, 'Ano de lancamento': released})

# By rating
df = df.sort_values(by=['Nota'], ascending=False)

df.to_csv(f'books_{genre}_goodreads.csv')